In [1]:
import MySQLdb
import pandas as pd
from tqdm import trange

In [2]:
df = pd.read_csv("/home/cosbi/financialSite/DB/line_memo.csv")
df = df.fillna("NULL")
df.drop_duplicates(inplace = True)
df.reset_index(drop = True, inplace = True) 

In [3]:
df

,stockNum,stockName,date,filePath,inputTime,username
0,9911.0,櫻花,2018-05-30,9911_櫻花_2018-05-30.txt,2:22:45 PM,popowing
1,9914.0,美利達,2018-05-30,9914_美利達_2018-05-30.txt,2:23:12 PM,popowing
2,3346.0,麗清,2018-05-30,3346_麗清_2018-05-30.txt,2:23:26 PM,popowing
3,2603.0,長榮,2018-05-30,2603_長榮_2018-05-30.txt,2:24:05 PM,popowing
4,8462.0,柏文,2018-05-30,8462_柏文_2018-05-30.txt,2:24:14 PM,popowing
...,...,...,...,...,...,...
438,1589.0,永冠-KY,2019-01-27,1589_永冠-KY_2019-01-27.txt,下午5:34:17,c21221203
439,5530.0,龍巖,2019-01-29,5530_龍巖_2019-01-29.txt,上午10:40:02,c21221203
440,3708.0,上緯,2019-01-30,3708_上緯_2019-01-30.txt,下午1:45:00,c21221203
441,2454.0,聯發科,2019-02-01,2454_聯發科_2019-02-01.txt,下午12:46:39,c21221203


In [4]:
db = MySQLdb.connect(host = "localhost", user = "debian-sys-maint",
                     passwd = "CEMj8ptYHraxNxFt", db = "financial", charset = "utf8")

cursor = db.cursor()

In [7]:
for i in trange(len(df)):
    cursor.execute("INSERT INTO lineMemo (stockNum, stockName, date, filePath, inputTime, username)"
                   " VALUES (%s, %s, %s, %s, %s, %s);", (str(df.iloc[i]["stockNum"]).replace(".0", ""), df.iloc[i]["stockName"], df.iloc[i]["date"].replace("-", "_"), "/home/cosbi/桌面/financialData/lineMemo_data/" + df.iloc[i]["filePath"], df.iloc[i]["inputTime"], df.iloc[i]["username"]))
    db.commit()

db.close()

100%|█████████████████████████████████████████| 443/443 [00:51<00:00,  8.62it/s]


In [12]:
db.close()

OperationalError: (2006, '')